# <font color='blue'>ALPAR - Governo Digital - Processo Inteligente</font>

# <font color='red'>recomendação de ações - gravando em MySQL</font>

In [1]:
import pandas as pd
import numpy as np
import copy
import csv
import codecs
import time
import datetime
from os.path import expanduser
import sqlite3

print("pandas versão", pd.__version__)
print("numpy versão", np.__version__)
print("csv versão", csv.__version__)

pd.options.display.max_rows = 2000
pd.options.display.width = 120
pd.options.display.max_colwidth = 100
pd.set_option("display.max_columns", None)

pandas versão 1.3.3
numpy versão 1.20.3
csv versão 1.0


# <font color='black'>trata o dataset total</font>

In [2]:
# escolhe o ambiente operacional
WIN  = True

# localização dos arquivos
if WIN:
    path_db       = 'C:/Users/Administrator/data-export/'
else:
    path_db       = '/Users/efz/DADOS_ALPAR_GD/'

In [3]:
# escolhe qual ambiente será carregado
# {'chave': ('nome do ambiente', 'in', 'out')}
ambientes = {
    0: ('produção',        'bd_fontes.sqlite',     'bd_reco.sqlite'),
    1: ('homologação',     'bd_fontes_hom.sqlite', 'bd_reco_hom.sqlite'),
    2: ('desenvolvimento', 'bd_fontes_dev.sqlite', 'bd_reco_dev.sqlite'),
    3: ('simulação',       'bd_fontes_sim.sqlite', 'bd_reco_sim.sqlite'),
} 

# selecione o banco em questão
amb = 1

ARQ_SQLITE       = ambientes[amb][1]
ARQ_SQLITE_RECO  = ambientes[amb][2]

In [4]:
def Tr_tasks_Antigo(df):
    tr = {
    'protocolo'                   :'protocolo',
    'entidade'                    :'entidade',
    'entityCode'                  :'Entidade - código',
    'servico'                     :'servico',
    'usuario'                     :'Usuário',
    'grupo'                       :'Grupo',
    'data_conclusao'              :'dthr_conclusao',
    'data_atendimento'            :'Data e Hora de atendimento',
    'data_criacao'                :'dthr_criacao',
    'acao'                        :'acao',
    'encaminhado_para'            :'encaminhado_para',
    'comentario'                  :'Comentário',
    'apoio'                       :'Apoio',
    'encerrado'                   :'proc_encerrado',
    'cancelado'                   :'proc_cancelado',
    'motivo_cancelamento'         :'Motivo de cancelamento',
    'notificacao'                 :'Notificação',
    'status_externo'              :'Status externo',
    'agendamento'                 :'Agendamento',
    'data_agendamento'            :'Data de agendamento',
    'categoria'                   :'Categoria',
    'grupo_responsavel'           :'Grupo responsável',
    'prazo'                       :'Prazo (em segundos)',
    }
    return df.rename(columns=tr)

In [5]:
conn = sqlite3.connect(path_db + ARQ_SQLITE)

# lê tabelas sqlite
print('Le_Dataset_SQLITE - Lendo tabelas do SQLITE...', end=' ')
sql_tasks = ("SELECT * FROM tasks")

df_tasks = pd.read_sql(sql_tasks,   conn, coerce_float=False)

# renomeia as colunas
df_tasks = Tr_tasks_Antigo(df_tasks)

print(f'tasks={df_tasks.shape} OK')

# fechar a conexão
conn.close()

Le_Dataset_SQLITE - Lendo tabelas do SQLITE... tasks=(1093, 26) OK


In [6]:
# coloca null em todas as colunas com dados vazios ou null
df_tasks['dthr_conclusao'] = df_tasks['dthr_conclusao'].replace(['null', '', '-'], np.nan)
df_tasks['proc_encerrado'] = df_tasks['proc_encerrado'].replace(['null', ''], '0').astype(np.int8)
df_tasks['proc_cancelado'] = df_tasks['proc_cancelado'].replace(['null', ''], '0').astype(np.int8)

# transforma datas do tipo string para tipo data
df_tasks['dthr_conclusao'] = pd.to_datetime(df_tasks['dthr_conclusao'])
df_tasks['dthr_criacao']   = pd.to_datetime(df_tasks['dthr_criacao'])

In [7]:
# agrupa por protocolo, obtem o máximo de encerrado e cancelado
df = df_tasks.groupby('protocolo').agg({'proc_encerrado': 'max', 'proc_cancelado': 'max'})
# display(df.sample(10))

In [8]:
# cria um dataframe de processos encerrados
lst_ence = list(df[(df['proc_encerrado'] == 1)].index)

# cria um dataframe de processos em andamento
lst_anda = list(df[ (df['proc_encerrado'] == 0) & (df['proc_cancelado'] == 0)].index)

# <font color='black'>cria e trata o dataset de protocolos encerrados</font>

In [9]:
#colunas que serão utilizadas
cols = ['id', 'protocolo', 'entidade', 'Entidade - código', 'servico', 'Usuário', 'Grupo', 'dthr_conclusao',
        'Data e Hora de atendimento', 'dthr_criacao', 'acao', 'encaminhado_para', 'Comentário', 'Apoio',
        'Motivo de cancelamento', 'Notificação', 'Status externo', 'Agendamento',
        'Data de agendamento', 'Categoria', 'Grupo responsável', 'Prazo (em segundos)', 'createdAt', 'updatedAt']
# cria dataset de tasks encerradas
df_ence = df_tasks[df_tasks['protocolo'].isin(lst_ence)][cols]
# display(df_ence.head(10))

In [10]:
# coloca a string <vazio> nos valores NULL
df_ence['encaminhado_para'].fillna('<vazio>', inplace=True)

In [11]:
# ordena por entidade, serviço, protocolo, dt_criação e recria o índice
df_ence = df_ence.sort_values(by=['entidade', 'servico', 'protocolo', 'dthr_criacao']).reset_index(drop=True)

In [12]:
# calcula a quantidade de dias da ação e exclui as datas de criação e de conclusão
df_ence['dias'] = (df_ence['dthr_conclusao'] - df_ence['dthr_criacao']) / np.timedelta64(1, 'D')
df_ence.drop(['dthr_conclusao', 'dthr_criacao'], axis=1, inplace=True)

In [13]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_ence = (df_ence.groupby(['protocolo', 'entidade', 'servico'])
      .agg(
        lst_acao=('acao', lambda x: tuple(x)),
        lst_encaminhado=('encaminhado_para', lambda x: tuple(x)),
        sum_dias=('dias', sum)).reset_index())

In [14]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_ence = (df_ence.groupby(['entidade', 'servico', 'lst_acao', 'lst_encaminhado'])
      .agg(media_dias=('sum_dias', 'mean')).reset_index())

# <font color='black'>cria o dataset de protocolos em andamento</font>

In [15]:
#colunas que serão utilizadas
cols = ['id', 'protocolo', 'entidade', 'Entidade - código', 'servico', 'Usuário', 'Grupo',
        'Data e Hora de atendimento', 'dthr_criacao', 'acao', 'encaminhado_para', 'Comentário', 'Apoio',
        'Motivo de cancelamento', 'Notificação', 'Status externo', 'Agendamento',
        'Data de agendamento', 'Categoria', 'Grupo responsável', 'Prazo (em segundos)', 'createdAt', 'updatedAt']
# cria dataset de tasks em andamento
df_anda = df_tasks[df_tasks['protocolo'].isin(lst_anda)][cols]

In [16]:
# exclui as linhas com a coluna ação NULL
df_anda = df_anda[ (df_anda['acao'] != '' ) & (df_anda['acao'].notna() ) ]

In [17]:
# ordena por entidade, serviço, protocolo, dt_criação
df_anda = df_anda.sort_values(by = ['entidade', 'servico', 'protocolo', 'dthr_criacao']).reset_index(drop=True)

In [18]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_anda = (df_anda.groupby(['protocolo', 'entidade', 'servico'])
      .agg(lst_acao=('acao', lambda x: tuple(x)))
      .reset_index()).copy()
df_anda.sort_values(['entidade', 'servico'], inplace = True)
# df_anda

# <font color='black'>gera estrutura de recomendações</font>

In [19]:
start = time.process_time()
tot = 0

# analisa cada protocolo em andamento
# procura a sequência do protocolo em andamento para sugerir o restante da sequência

# zera a lista de recomendações
lst_reco = []
const_max_tempo = 999999999.9
tot_anda = range(len(df_anda))
# tot_anda = range(1000)
print('total de protocolos a processar =', tot_anda)

# para cada protocolo em andamento procura uma coleção de sequências dos protocolos encerrados
anda_quebra = [None, None]
for i in tot_anda:
    if anda_quebra != [df_anda.loc[i, 'entidade'], df_anda.loc[i, 'servico']]:
        anda_quebra = [df_anda.loc[i, 'entidade'], df_anda.loc[i, 'servico']]
        df_aux1 = df_ence[(df_ence['entidade'] == df_anda.loc[i, 'entidade']) & 
                          (df_ence['servico'] == df_anda.loc[i, 'servico'])]
    
    n_acoes = len(df_anda.loc[i, 'lst_acao'])
    df_aux = df_aux1[(df_aux1['lst_acao'].apply(lambda x: x[:n_acoes]) == df_anda.loc[i, 'lst_acao'])]
    
    df_aux.reset_index(inplace = True)
    
    # zera a variável que vai achar o registro "entidade/serviço" com o menor tempo de conclusão
    # para quando for achado mais de 1 possibilidade
    min_tempo = const_max_tempo
    dt = {'sequencia': [], 'encaminhamento': []}

    qtd_casos = len(df_aux)
    tot += qtd_casos
        
    for k in range(qtd_casos):
        valor = round(np.float64(df_aux.loc[k, 'media_dias']), 6)
        if (valor < min_tempo) or (min_tempo == None):
            min_tempo = valor
            dt = {'sequencia': list(df_aux.loc[k, 'lst_acao']),
                  'encaminhamento': list(df_aux.loc[k, 'lst_encaminhado'])}
            
    indice = len(list(df_anda.loc[i, 'lst_acao'])) if dt else -1
    if qtd_casos == 1:
        tipo_recomendacao = 2
    elif qtd_casos > 1:
        tipo_recomendacao = 3
    else:
        tipo_recomendacao = 1
        
    it = {'protocolo': df_anda.loc[i, 'protocolo'],
            'seq_atual': list(df_anda.loc[i, 'lst_acao']),
            'recomendacao': dt,
            'tempo_medio': None if min_tempo == const_max_tempo else min_tempo,
            'reco_acao_idx': indice,
            'qtd_casos' : qtd_casos,
            'tipo_recomendacao' : tipo_recomendacao}
    lst_reco.append(it)
    if i % 100 == 0:
        print(i, tot)

print(i, tot)
print('tempo =', time.process_time() - start)

total de protocolos a processar = range(0, 531)
0 1
100 20
200 61
300 103
400 104
500 107
530 123
tempo = 0.875


# <font color='black'>cria um arquivo com recomendações</font>

## <font color='green'>CRIA as LISTAS de gravação de protocolos e recomendações no MySQL</font>

In [20]:
# prepara uma lista de tuplas para gravação da tabela "tb_protocolo" no MySQL
lst_protocolo = []
lst_recomenda = []
cont_r = 0
for k in lst_reco:
    prx_acao = None
    prx_encaminhamento = None
    
    n_acoes_ence = len(k['recomendacao']['sequencia'])
    for j in range(n_acoes_ence):
        cont_r += 1
        
        if j == k['reco_acao_idx']:
            proximo = 1
            prx_acao = k['recomendacao']['sequencia'][j]
            prx_encaminhamento = k['recomendacao']['encaminhamento'][j]
        else:
            proximo = 0
        
        lst_recomenda.append((cont_r,
                              j, 
                              k['protocolo'], 
                              k['recomendacao']['sequencia'][j], 
                              k['recomendacao']['encaminhamento'][j], 
                              proximo
                             )
        )

    lst_protocolo.append((k['protocolo'], 
                          k['tempo_medio'], 
                          prx_acao, 
                          prx_encaminhamento,
                          datetime.datetime.now(),
                          k['qtd_casos'],
                          k['tipo_recomendacao']
                         )
    )
cols_p = ['pk_protocolo', 'tp_medio', 'nm_prox_acao', 'nm_prox_enc', 'dt_timestamp', 'cd_tipo_reco', 'nu_qtd_casos']
cols_r = ['sq_acao', 'nu_seq', 'fk_protocolo', 'nm_acao', 'nm_encaminhado', 'fg_proximo']
df_protocolo = pd.DataFrame(lst_protocolo, columns=cols_p)
df_recomenda = pd.DataFrame(lst_recomenda, columns=cols_r)

# display(df_protocolo)
# display(df_recomenda)


## <font color='green'>GRAVA as tabelas de protocolos e recomendações no MySQL</font>

In [21]:
# import the module
print('LoadBD - carregando tabelas ... ')

try:
    conn = sqlite3.connect(path_db + ARQ_SQLITE_RECO)
    
    print('    carregando tabela tb_protocolo ... ', end='')
    df_protocolo.to_sql('tb_protocolo', con=conn, if_exists='replace', index=False, chunksize = 10000)
    print('OK')

    print('    carregando tabela tb_recomenda ... ', end='')
    df_recomenda.to_sql('tb_recomenda', con=conn, if_exists='replace', index=False, chunksize = 10000)
    print('OK')
    
except ValueError as vx:
    print('ERROR -', vx)

except Exception as ex: 
    print('EXCEPTION -', ex)

else:
    print('Tabelas criadas com sucesso');  

finally:
    conn.close()


LoadBD - carregando tabelas ... 
    carregando tabela tb_protocolo ... OK
    carregando tabela tb_recomenda ... OK
Tabelas criadas com sucesso
